# Analyse Bouwgegevens per Gewest

Deze notebook toont de evolutie van renovaties en nieuwbouw voor de drie gewesten in België:
- Vlaams Gewest
- Waals Gewest  
- Brussels Hoofdstedelijk Gewest

Voor elke grafiek worden de maandelijkse gegevens getoond als stippellijn en het 12-maands voortschrijdend gemiddelde als volle lijn.

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configure plotly to show charts in notebook
import plotly.io as pio
pio.renderers.default = "notebook"

In [ ]:
# Load the data
renovatie_df = pd.read_csv('data/csv/Renovatie_per_bestemming.csv')
nieuwbouw_df = pd.read_csv('data/csv/Bouwvergunningen_voor_woongebouwen,_indeling_naar_arrondissementen.csv')

print("Renovatie data shape:", renovatie_df.shape)
print("Nieuwbouw data shape:", nieuwbouw_df.shape)
print("\nRenovatie columns:", renovatie_df.columns.tolist())
print("Nieuwbouw columns:", nieuwbouw_df.columns.tolist())

# Check unique regions in both datasets
print("\nUnique regions in renovatie data:")
print(renovatie_df['regio'].unique())
print("\nUnique regions in nieuwbouw data:")
print(nieuwbouw_df['regio'].unique())

In [ ]:
# Filter nieuwbouw data to only include the three main regions
gewesten = ['VLAAMS GEWEST', 'WAALS GEWEST', 'BRUSSELS HOOFDSTEDELIJK GEWEST']
nieuwbouw_filtered = nieuwbouw_df[nieuwbouw_df['regio'].isin(gewesten)].copy()

# Convert month names to numbers for proper sorting
month_mapping = {
    'Januari': 1, 'Februari': 2, 'Maart': 3, 'April': 4, 'Mei': 5, 'Juni': 6,
    'Juli': 7, 'Augustus': 8, 'September': 9, 'Oktober': 10, 'November': 11, 'December': 12
}

# Prepare nieuwbouw data
nieuwbouw_filtered['maand_num'] = nieuwbouw_filtered['maand'].map(month_mapping)
nieuwbouw_filtered['date'] = pd.to_datetime(dict(year=nieuwbouw_filtered['jaar'], 
                                                 month=nieuwbouw_filtered['maand_num'], 
                                                 day=1))
nieuwbouw_agg = nieuwbouw_filtered.groupby(['regio', 'date'])['aantal woningen'].sum().reset_index()

# Prepare renovatie data - aggregate by region, year, month
renovatie_df['date'] = pd.to_datetime(dict(year=renovatie_df['jaar'], 
                                          month=renovatie_df['maand'], 
                                          day=1))
renovatie_agg = renovatie_df.groupby(['regio', 'date'])['woningen'].sum().reset_index()

print("Nieuwbouw aggregated data shape:", nieuwbouw_agg.shape)
print("Renovatie aggregated data shape:", renovatie_agg.shape)
print("\nDate range for nieuwbouw:", nieuwbouw_agg['date'].min(), "to", nieuwbouw_agg['date'].max())
print("Date range for renovatie:", renovatie_agg['date'].min(), "to", renovatie_agg['date'].max())

In [ ]:
def calculate_moving_average(df, value_col, date_col='date', window=12):
    """Calculate 12-month moving average for each region"""
    df_sorted = df.sort_values(['regio', date_col])
    df_sorted[f'{value_col}_ma'] = df_sorted.groupby('regio')[value_col].rolling(window=window, min_periods=1).mean().reset_index(0, drop=True)
    return df_sorted

def create_chart(data, value_col, title_type, y_axis_title, gewest_name):
    """Create a plotly chart with monthly data and moving average using Flemish number formatting"""
    
    # Define color for this gewest
    colors = {
        'VLAAMS GEWEST': '#1f77b4',
        'WAALS GEWEST': '#ff7f0e', 
        'BRUSSELS HOOFDSTEDELIJK GEWEST': '#2ca02c'
    }
    
    color = colors.get(gewest_name, '#d62728')
    
    # Filter data for this gewest
    gewest_data = data[data['regio'] == gewest_name].sort_values('date')
    
    # Get date range for the title
    start_date = gewest_data['date'].min().strftime('%Y')
    end_date = gewest_data['date'].max().strftime('%B %Y')
    
    # Create comprehensive, publication-ready titles with correct Dutch capitalization
    if title_type == 'nieuwbouw':
        main_title = f"Evolutie bouwvergunningen voor nieuwbouw woongebouwen"
        subtitle = f"{gewest_name} ({start_date} - {end_date})"
        source_info = "Bron: Statbel, bouwvergunningen voor woongebouwen per arrondissement | Verwerking: Embuild Vlaanderen"
    else:  # renovatie
        main_title = f"Evolutie renovatievergunningen voor woongebouwen"
        subtitle = f"{gewest_name} ({start_date} - {end_date})"
        source_info = "Bron: Statbel, renovaties per bestemming | Verwerking: Embuild Vlaanderen"
    
    # Title without source info (source will be added as annotation below chart)
    chart_title = f"<b>{main_title}</b><br><sub>{subtitle}</sub>"
    
    # Create custom hover text with Flemish formatting
    monthly_hover_text = []
    ma_hover_text = []
    
    for _, row in gewest_data.iterrows():
        date_str = row['date'].strftime('%B %Y')  # More readable date format
        monthly_val = format_flemish_number(row[value_col], 0)
        ma_val = format_flemish_number(row[f'{value_col}_ma'], 1)
        
        monthly_hover_text.append(f'<b>Maandelijks</b><br>Periode: {date_str}<br>{y_axis_title}: {monthly_val}')
        ma_hover_text.append(f'<b>12-maands voortschrijdend gemiddelde</b><br>Periode: {date_str}<br>{y_axis_title}: {ma_val}')
    
    fig = go.Figure()
    
    # Add monthly data (dashed line)
    fig.add_trace(go.Scatter(
        x=gewest_data['date'],
        y=gewest_data[value_col],
        mode='lines',
        name=f'Maandelijks',
        line=dict(color=color, dash='dash', width=2),
        hovertemplate='%{text}<extra></extra>',
        text=monthly_hover_text
    ))
    
    # Add 12-month moving average (solid line)
    fig.add_trace(go.Scatter(
        x=gewest_data['date'],
        y=gewest_data[f'{value_col}_ma'],
        mode='lines',
        name=f'12-maands voortschrijdend gemiddelde',
        line=dict(color=color, width=4),
        hovertemplate='%{text}<extra></extra>',
        text=ma_hover_text
    ))
    
    fig.update_layout(
        title=dict(
            text=chart_title,
            x=0.5,
            font=dict(size=14),
            pad=dict(t=20, b=20)
        ),
        xaxis_title='<b>Periode</b>',
        yaxis_title=f'<b>{y_axis_title}</b>',
        hovermode='x unified',
        template='plotly_white',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5,
            font=dict(size=11)
        ),
        height=520,  # Extra height for source annotation below
        margin=dict(t=100, b=80, l=50, r=50),  # More bottom margin for source
        plot_bgcolor='rgba(248,249,250,0.8)',
        font=dict(family="Arial, sans-serif")
    )
    
    # Add source information as annotation below the chart
    fig.add_annotation(
        text=f"<i>{source_info}</i>",
        xref="paper", yref="paper",
        x=0.5, y=-0.12,  # Position below the chart
        xanchor='center', yanchor='top',
        showarrow=False,
        font=dict(size=9, color="#666666"),
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="rgba(200,200,200,0.5)",
        borderwidth=1
    )
    
    # Add subtle grid for better readability
    fig.update_xaxes(
        showgrid=True, 
        gridwidth=1, 
        gridcolor='rgba(128,128,128,0.2)',
        tickformat='%Y'
    )
    fig.update_yaxes(
        showgrid=True, 
        gridwidth=1, 
        gridcolor='rgba(128,128,128,0.2)',
        tickformat=',',
        separatethousands=True
    )
    
    return fig

# Calculate moving averages
nieuwbouw_with_ma = calculate_moving_average(nieuwbouw_agg, 'aantal woningen')
renovatie_with_ma = calculate_moving_average(renovatie_agg, 'woningen')

In [ ]:
def format_flemish_number(number, decimals=0):
    """Format number using Flemish notation: periods for thousands, commas for decimals"""
    if pd.isna(number):
        return ""
    
    if decimals == 0:
        # For integers, format with periods as thousands separators
        return f"{int(number):,}".replace(",", ".")
    else:
        # For decimals, format with periods for thousands and comma for decimal
        formatted = f"{number:,.{decimals}f}"
        # Replace comma (thousands) with periods and dot (decimal) with comma
        return formatted.replace(",", "X").replace(".", ",").replace("X", ".")

def create_data_table(data, gewest_name, data_type):
    """Create a scrollable HTML table for the data with Flemish number formatting"""
    from IPython.display import HTML
    
    # Filter data for this gewest
    gewest_data = data[data['regio'] == gewest_name].sort_values('date', ascending=False)
    
    # Format the data for display
    display_data = gewest_data.copy()
    display_data['Datum'] = display_data['date'].dt.strftime('%d-%m-%Y')  # Also use Flemish date format
    
    if data_type == 'nieuwbouw':
        display_data['Aantal Woningen'] = display_data['aantal woningen'].apply(lambda x: format_flemish_number(x, 0))
        display_data['12-maands Gemiddelde'] = display_data['aantal woningen_ma'].apply(lambda x: format_flemish_number(x, 1))
        value_cols = ['Datum', 'Aantal Woningen', '12-maands Gemiddelde']
    else:  # renovatie
        display_data['Aantal Woningen'] = display_data['woningen'].apply(lambda x: format_flemish_number(x, 0))
        display_data['12-maands Gemiddelde'] = display_data['woningen_ma'].apply(lambda x: format_flemish_number(x, 1))
        value_cols = ['Datum', 'Aantal Woningen', '12-maands Gemiddelde']
    
    # Create HTML table
    table_html = f"""
    <div style="margin-top: 20px;">
        <h4>{data_type.title()} Data - {gewest_name}</h4>
        <div style="max-height: 400px; overflow-y: auto; border: 1px solid #ddd;">
            <table style="width: 100%; border-collapse: collapse; font-family: Arial, sans-serif;">
                <thead style="background-color: #f2f2f2; position: sticky; top: 0;">
                    <tr>
    """
    
    for col in value_cols:
        table_html += f'<th style="border: 1px solid #ddd; padding: 8px; text-align: left;">{col}</th>'
    
    table_html += """
                    </tr>
                </thead>
                <tbody>
    """
    
    for _, row in display_data[value_cols].iterrows():
        table_html += "<tr>"
        for col in value_cols:
            align = "right" if col != "Datum" else "left"  # Right-align numbers, left-align dates
            table_html += f'<td style="border: 1px solid #ddd; padding: 8px; text-align: {align};">{row[col]}</td>'
        table_html += "</tr>"
    
    table_html += """
                </tbody>
            </table>
        </div>
        <p style="font-size: 12px; color: #666; margin-top: 10px;">
            <em>Tabel toont de meest recente data bovenaan. Scroll om oudere gegevens te bekijken. Getallen volgen Vlaamse notatie (punten voor duizendtallen, komma's voor decimalen).</em>
        </p>
    </div>
    """
    
    return HTML(table_html)

## Vlaams Gewest

### Nieuwbouw en renovaties in het Vlaams Gewest

In [ ]:
# Nieuwbouw - Vlaams Gewest
fig_vlaams_nieuwbouw = create_chart(
    nieuwbouw_with_ma, 
    'aantal woningen', 
    'nieuwbouw',  # Updated parameter
    'Aantal Woningen',
    'VLAAMS GEWEST'
)
fig_vlaams_nieuwbouw.show()

# Display data table
display(create_data_table(nieuwbouw_with_ma, 'VLAAMS GEWEST', 'nieuwbouw'))

In [ ]:
# Renovatie - Vlaams Gewest
fig_vlaams_renovatie = create_chart(
    renovatie_with_ma, 
    'woningen', 
    'renovatie',  # Updated parameter
    'Aantal Woningen',
    'VLAAMS GEWEST'
)
fig_vlaams_renovatie.show()

# Display data table
display(create_data_table(renovatie_with_ma, 'VLAAMS GEWEST', 'renovatie'))

## Waals Gewest

### Nieuwbouw en Renovaties in het Waals Gewest

In [ ]:
# Nieuwbouw - Waals Gewest
fig_waals_nieuwbouw = create_chart(
    nieuwbouw_with_ma, 
    'aantal woningen', 
    'nieuwbouw',  # Updated parameter
    'Aantal Woningen',
    'WAALS GEWEST'
)
fig_waals_nieuwbouw.show()

# Display data table
display(create_data_table(nieuwbouw_with_ma, 'WAALS GEWEST', 'nieuwbouw'))

In [ ]:
# Renovatie - Waals Gewest
fig_waals_renovatie = create_chart(
    renovatie_with_ma, 
    'woningen', 
    'renovatie',  # Updated parameter
    'Aantal Woningen',
    'WAALS GEWEST'
)
fig_waals_renovatie.show()

# Display data table
display(create_data_table(renovatie_with_ma, 'WAALS GEWEST', 'renovatie'))

## Brussels Hoofdstedelijk Gewest

### Nieuwbouw en Renovaties in het Brussels Hoofdstedelijk Gewest

In [ ]:
# Nieuwbouw - Brussels Hoofdstedelijk Gewest
fig_brussel_nieuwbouw = create_chart(
    nieuwbouw_with_ma, 
    'aantal woningen', 
    'nieuwbouw',  # Updated parameter
    'Aantal Woningen',
    'BRUSSELS HOOFDSTEDELIJK GEWEST'
)
fig_brussel_nieuwbouw.show()

# Display data table
display(create_data_table(nieuwbouw_with_ma, 'BRUSSELS HOOFDSTEDELIJK GEWEST', 'nieuwbouw'))

In [ ]:
# Renovatie - Brussels Hoofdstedelijk Gewest
fig_brussel_renovatie = create_chart(
    renovatie_with_ma, 
    'woningen', 
    'renovatie',  # Updated parameter
    'Aantal Woningen',
    'BRUSSELS HOOFDSTEDELIJK GEWEST'
)
fig_brussel_renovatie.show()

# Display data table
display(create_data_table(renovatie_with_ma, 'BRUSSELS HOOFDSTEDELIJK GEWEST', 'renovatie'))